In [ ]:
import numpy as np
import pandas as pd

import requests

In [4]:
df = pd.read_excel("test.xlsx", sheet_name="Sheet3")
df.head()

,Month,Forecast
0,2026-01-01,6360
1,2026-02-01,17280
2,2026-03-01,17820
3,2026-04-01,24240
4,2026-05-01,33060


In [8]:
df[df['Forecast']==df['Forecast'].max()]

,Month,Forecast
34,2028-11-01,173460
